In [1]:
%pip install mlx
%pip install numpy
%pip install opencv-python


[notice] A new release of pip is available: 23.2.1 -> 23.3.2
[notice] To update, run: python3.11 -m pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.

[notice] A new release of pip is available: 23.2.1 -> 23.3.2
[notice] To update, run: python3.11 -m pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [2]:
import numpy as np
import mlx.core as mx
import time
import cv2
import os

In [3]:
### MLX

In [4]:
const255mx = mx.array(255.0, mx.float32) #255.0
const128mx = mx.array(128.0, mx.float32) # 128.0

In [5]:
def darken_blend_8_mx(base, active):
  return mx.where(mx.greater(base, active), active, base)

In [6]:
def color_burn_8_mx(base, active):
  return mx.where(mx.equal(active, 0), const255mx, const255mx - (const255mx - base) / active)

In [7]:
def lighten_blend_8_mx(base, active):
  return mx.where(mx.less(base, active), active, base)

In [8]:
def color_dodge_8_mx(base, active):
  return mx.where(mx.equal(active, const255mx), const255mx, base / (const255mx - active))

In [9]:
def overlay_blend_8_mx(base, active):
  return mx.where(mx.greater_equal(base, const128mx), 2 * base + base - 2 * base * base / const255mx - const255mx, 2 * base * base / const255mx)

In [10]:
def multiply_blend_8_mx(base, active):
  return base * active / const255mx

In [11]:
def linear_burn_8_mx(base, active):
  return base + active - const255mx

In [12]:
def screen_blend_8_mx(base, active):
  return base + active - base * active / const255mx

In [13]:
def linear_dodge_8_mx(base, active):
  return base + active

In [14]:
### Single

In [15]:
def normal_blend_f_mx(base, active, opacity):
  return opacity * active + (1-opacity) * base

In [16]:
def normal_blend_8_mx(base, active, opacity):
  return opacity * active + (const255mx - opacity) * base

In [17]:
### Runner

In [18]:
size = 512
np.random.seed(1)

list1 = np.random.rand(size).astype('float32')
list2 = np.random.rand(size).astype('float32')
list3 = np.random.rand(size)
list1 = mx.array(list1, mx.float32)
list2 = mx.array(list2, mx.float32)

int1 = mx.array(np.random.rand(1).astype('float32'), mx.float32)

In [19]:
folder = "./data/"

img_files = [os.path.join(folder, f) for f in os.listdir(folder) if os.path.isfile(os.path.join(folder, f))]

bases = []
actives = []

for _file in img_files:
    img = cv2.imread(_file, cv2.IMREAD_GRAYSCALE)
    rnd = np.random.rand(*img.shape).astype('float32')

    bases.append(img)
    actives.append(rnd)

In [20]:
def runner(bases, actives, f):
    total_time = 0
    for i in range(len(bases)):
        b = mx.array(bases[i], mx.float32)
        a = mx.array(actives[i], mx.float32)
        start_time = time.perf_counter()
        mx.eval(f(b, a))
        end_time = time.perf_counter()
        del a
        del b
        total_time += (end_time - start_time) * 1000
    return total_time

def mat_timer(bases, actives, f):
    runs = 5
    times = []
    for _ in range(runs):
        times.append(runner(bases, actives, f))
    times = np.array(times)
    print(f"{np.average(times)}ms +/- {np.std(times)}ms")

In [21]:
mat_timer(bases, actives, darken_blend_8_mx)

6414.046255592257ms +/- 374.5294325176546ms


In [22]:
mat_timer(bases, actives, color_burn_8_mx)

8064.282566960901ms +/- 295.1573672833318ms


In [23]:
mat_timer(bases, actives, lighten_blend_8_mx)

7028.574208542705ms +/- 402.2918122230905ms


In [24]:
mat_timer(bases, actives, color_dodge_8_mx)

8230.475757271051ms +/- 129.34228822621841ms


In [25]:
mat_timer(bases, actives, overlay_blend_8_mx)

10926.442891173065ms +/- 220.33721278313888ms


In [26]:
mat_timer(bases, actives, multiply_blend_8_mx) 

7983.681849669665ms +/- 685.2647212963493ms


In [27]:
mat_timer(bases, actives, linear_burn_8_mx)

7680.3849728778005ms +/- 653.2197472203173ms


In [28]:
mat_timer(bases, actives, screen_blend_8_mx)

8242.720625549555ms +/- 212.17419665883756ms


In [29]:
mat_timer(bases, actives, linear_dodge_8_mx)

7272.81727110967ms +/- 341.0646680882315ms


In [30]:
%timeit list3 = mx.eval(normal_blend_f_mx(list1, list2, int1))

167 µs ± 16.7 µs per loop (mean ± std. dev. of 7 runs, 1,000 loops each)


In [31]:
%timeit list3 = mx.eval(normal_blend_8_mx(list1, list2, int1))

194 µs ± 24.7 µs per loop (mean ± std. dev. of 7 runs, 10,000 loops each)
